In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# seed 고정
import random
import os
import numpy as np

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42)

In [ ]:
# 경고 메시지 무시
import warnings

warnings.filterwarnings('ignore')

In [ ]:
# 필요한 라이브러리 설치
!pip install transformers pandas scikit-learn
!pip install adabelief-pytorch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
# 필요한 라이브러리
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
import torch

#### 데이터 불러오기 및 전처리

In [ ]:
# 데이터 로드 및 전처리
data = pd.read_csv("/content/drive/MyDrive/포트폴리오/정보관리학회 논문/book.csv", encoding = 'utf-8-sig')

data.head()

,메타데이터,자료상태,연구과제번호,결과물 구분,자료유형,저작,서명,주초록,메타작성자초록,목차,주제용어,용어사전,주제분류
0,4b76f6df51e1f6710152205f297720e2,3차검증완료,2012-S1A5B5A02-2012S1A5B5A02023627,원자료,단행본,"중세 이성의 종교평화 기획: 아벨라르두스, 룰루스, 쿠사누스의 종교간 대화 문헌 연구",[단행본] Sic et non : a critical edition,NaN,본 자료는 최종결과물(결과보고서)의 원자료이다.\n\n아벨라르두스의 변증법을 통해 ...,NaN,"- 언어: 라틴어, - 페이지 수 확인이 어려움",종교 종교 간 대화 평화 관용 유일신교 그리스도교 유대교 이슬람교 삼위일체 자연법 ...,"▣ 학진주제분류, 논리학/논리철학, 기타수학"
1,4b76f6df51e1f6710152209193d32174,3차검증완료,2012-S1A5B5A07-2012S1A5B5A07034975,원자료,단행본,"쇼펜하우어와 바타이유에서인간이성의 양면성과 전쟁의 근원 -쇼펜하우어, 바타이유, 셸...",[단행본] (레비나스)타자의 윤리학,NaN,본 자료는 최종결과물(결과보고서)의 원자료이다.\n\n레비나스 타자의 철학을 중심으...,책머리에 = 6\n타자윤리의 개요 = 12\n제1장 타자윤리학의 기초 = 21\n1...,NaN,전쟁 감정 이성 충동 레비나스,"▣ 학진주제분류, 철학일반, 서양근대프랑스철학"
2,4b76f6df51e1f6710152222ecc3621c8,3차검증완료,2012-S1A5A8-2012S1A5A8022449,원자료,단행본,괴테와 죽음의 현상학 - 괴테의 작품에 나타난 죽음서사 분석과 웰다잉(죽음대비)교육...,[단행본] 친화력,부유한 남작 에두아르트와 그의 부인 샤로테는 서로의 뜻과는 달리 각기 다른 사람과 ...,본 자료는 최종결과물(결과보고서)의 원자료이다.\n\n본 도서는 괴테의 개별작품 안...,제1부 ...7\n제2부 ...157\n해설. 인간 관계의 실험실로서의 소설 ...321,"주초록, 목차 출처 : 인터넷 교보문고(http://www.kyobobook.co....",괴테 연애 소설 친화력 인간관계,"▣ 학진주제분류, 독일어와문학, 소설(독일문학), 교육학, 사회/문화심리, 사회문제심리"
3,4b76f6df522e37f901522f8a03f102be,3차검증완료,2012-S1A5B5A07-2012S1A5B5A07035681,원자료,단행본,"환대와 글쓰기 - 레비나스, 데리다, 자베스를 중심으로","[단행본] Totalité et infini, essai sur l'extériorite",« On conviendra aisément qu'il importe au plus...,본 자료는 최종결과물 (결과보고서)의 원자료이다.\n\n연구자는 본 자료를 활용하여...,NaN,NaN,환대 타자 타자성 엠마누엘 레비나스 자크 데리다 에드몽 자베스 무조건적 환대 문지방...,NaN
4,4b76f6df522e37f901522f8e3d2702ce,3차검증완료,2012-S1A5B5A07-2012S1A5B5A07035681,원자료,단행본,"환대와 글쓰기 - 레비나스, 데리다, 자베스를 중심으로",[단행본] Autrement qu'être ou au-delà de l'essence,NaN,본 자료는 최종결과물 (결과보고서)의 원자료이다.\n\n연구자는 본 자료를 활용하여...,NaN,NaN,환대 타자 타자성 엠마누엘 레비나스 자크 데리다 에드몽 자베스 무조건적 환대 문지방...,NaN


In [ ]:
# 소분류 레이블 데이터 불러오기
import pandas as pd

label = pd.read_csv("/content/drive/MyDrive/포트폴리오/정보관리학회 논문/label_processed.csv", encoding = 'cp949')

label.head()

,분야코드,구분,대분류명,중분류명,소분류명
0,A000000,A,인문학,인문학,인문학
1,A010000,A,인문학,사전학,사전학
2,A020000,A,인문학,역사학,역사학
3,A020100,A,인문학,역사학,역사일반
4,A020200,A,인문학,역사학,한국사


In [ ]:
# 대분류 중 '인문학'과 '사회과학' 선택
label = label[label['대분류명'].isin(['인문학', '사회과학'])]

# 소분류명이 '인문학', '사회과학' 삭제
label = label[~label['중분류명'].isin(['인문학', '사회과학'])]

# 소분류 분류 전처리
소분류_목록 = set(label['소분류명'])

In [ ]:
def filter_exact_match(row, 소분류_목록):
    # 주제분류 컬럼의 값을 소분류 목록과 정확히 일치하는 경우만 남기고 중복 제거
    filtered = [sub for sub in row.split(', ') if sub in 소분류_목록]
    return ', '.join(sorted(set(filtered), key=filtered.index))

# 주제분류 칼럼에서 결측값이 있는 행을 삭제
data = data.dropna(subset=['주제분류'])

# '주제분류' 컬럼의 값이 소분류 목록에 정확히 일치하는 경우만 남기고 중복 제거
data['주제분류'] = data['주제분류'].apply(lambda x: filter_exact_match(x, 소분류_목록))

# 중분류 목록에 포함된 단어가 없는 행 제거
data = data[data['주제분류'] != '']

In [ ]:
def get_last_word(text):
    if isinstance(text, str):  # 문자열인지 확인
        words = text.split()  # 공백으로 단어 분리
        return words[-1] if words else text  # 마지막 단어 반환
    return text  # 문자열이 아닌 경우 원래 값 반환

data['주제분류'] = data['주제분류'].apply(get_last_word)

data.head()

,메타데이터,자료상태,연구과제번호,결과물 구분,자료유형,저작,서명,주초록,메타작성자초록,목차,주제용어,용어사전,주제분류
1,4b76f6df51e1f6710152209193d32174,3차검증완료,2012-S1A5B5A07-2012S1A5B5A07034975,원자료,단행본,"쇼펜하우어와 바타이유에서인간이성의 양면성과 전쟁의 근원 -쇼펜하우어, 바타이유, 셸...",[단행본] (레비나스)타자의 윤리학,NaN,본 자료는 최종결과물(결과보고서)의 원자료이다.\n\n레비나스 타자의 철학을 중심으...,책머리에 = 6\n타자윤리의 개요 = 12\n제1장 타자윤리학의 기초 = 21\n1...,NaN,전쟁 감정 이성 충동 레비나스,철학일반
2,4b76f6df51e1f6710152222ecc3621c8,3차검증완료,2012-S1A5A8-2012S1A5A8022449,원자료,단행본,괴테와 죽음의 현상학 - 괴테의 작품에 나타난 죽음서사 분석과 웰다잉(죽음대비)교육...,[단행본] 친화력,부유한 남작 에두아르트와 그의 부인 샤로테는 서로의 뜻과는 달리 각기 다른 사람과 ...,본 자료는 최종결과물(결과보고서)의 원자료이다.\n\n본 도서는 괴테의 개별작품 안...,제1부 ...7\n제2부 ...157\n해설. 인간 관계의 실험실로서의 소설 ...321,"주초록, 목차 출처 : 인터넷 교보문고(http://www.kyobobook.co....",괴테 연애 소설 친화력 인간관계,사회문제심리
5,4b76f6df51e1f67101520c91c0be0eb0,3차검증완료,2012-S1A5B5A01-2012S1A5B5A01022174,원자료,단행본,미르체아 엘리아데 연작소설에서 나타난 청년담론과 문화·정치적 역할 1년차-‘훌리거니...,[단행본] Nuntă în cer,NaN,본 자료는 최종결과물(결과보고서)의 원자료이다.\n\n이 자료는 엘리아데의 연작소설...,NaN,NaN,미르체아 엘리아데 문학 율리시스 역사의 공포 크리테리온 청년담론 훌리거니즘 사랑 디...,루마니아어문학
6,4b76f6df51e1f67101520c9465d80eb7,3차검증완료,2012-S1A5B5A01-2012S1A5B5A01022174,원자료,단행본,미르체아 엘리아데 연작소설에서 나타난 청년담론과 문화·정치적 역할 1년차-‘훌리거니...,[단행본] Huliganii,NaN,본 자료는 최종결과물(결과보고서)의 원자료이다.\n\n자료는 엘리아데의 연작소설 작...,NaN,NaN,미르체아 엘리아데 문학 율리시스 역사의 공포 크리테리온 청년담론 훌리거니즘 사랑 디...,루마니아어문학
7,4b76f6df51e1f67101520c9f70940ec6,3차검증완료,2012-S1A5B5A07-2012S1A5B5A07035015,원자료,단행본,레오나르도 샤샤의 추리소설 형식과 비탈리아노 브란카티의 욕망 모티프에 나타난 문학성 비교,[단행본] Don Giovanni in Sicilia,NaN,본 자료는 최종결과물(결과보고서)의 원자료이다.\n\n자료는 브랑카티의 문학 소설작...,NaN,NaN,비탈리아노 브란카티 레오나르도 샤샤 욕망 갈리즘 추리소설 남부이탈리아 파시즘,이탈리아어문학


In [ ]:
data['자료상태'].value_counts()

,count
자료상태,
3차검증완료,13384
4차검증완료,129


In [ ]:
# 3차검증완료 이상만 추출
data = data[(data['자료상태'] == '3차검증완료') | (data['자료상태'] == '4차검증완료')]

data['자료상태'].value_counts()

,count
자료상태,
3차검증완료,13384
4차검증완료,129


In [ ]:
# 주제분류 칼럼 전처리
value_counts = data['주제분류'].value_counts()
to_keep = value_counts[value_counts >= 10].index
data = data[data['주제분류'].isin(to_keep)]

data['주제분류'].value_counts()

,count
주제분류,
국문학,1695
중문학,891
영문학,855
서양철학,850
일본문학,554
...,...
방언,10
인종및민족,10
의미론(언어학),10


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13008 entries, 1 to 17093
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   메타데이터     13008 non-null  object
 1   자료상태      13008 non-null  object
 2   연구과제번호    13008 non-null  object
 3   결과물 구분    12817 non-null  object
 4   자료유형      13008 non-null  object
 5   저작        13008 non-null  object
 6   서명        13008 non-null  object
 7   주초록       7759 non-null   object
 8   메타작성자초록   12303 non-null  object
 9   목차        7387 non-null   object
 10  주제용어      5241 non-null   object
 11  용어사전      13008 non-null  object
 12  주제분류      13008 non-null  object
dtypes: object(13)
memory usage: 1.4+ MB


In [ ]:
# 합칠 칼럼 목록 지정
columns_join = ['저작',	'서명', '주초록', '메타작성자초록 ', '목차', '주제용어', '용어사전']

# 전체 칼럼들을 하나의 칼럼으로 합치기: 결측값은 무시
data['combined_text'] = data[columns_join].apply(lambda row: ' '.join(row.dropna().values.astype(str)), axis=1)

# 통합텍스트 칼럼에서 결측값이 있는 행을 삭제
data = data.dropna(subset=['combined_text'])

data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13008 entries, 1 to 17093
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   메타데이터          13008 non-null  object
 1   자료상태           13008 non-null  object
 2   연구과제번호         13008 non-null  object
 3   결과물 구분         12817 non-null  object
 4   자료유형           13008 non-null  object
 5   저작             13008 non-null  object
 6   서명             13008 non-null  object
 7   주초록            7759 non-null   object
 8   메타작성자초록        12303 non-null  object
 9   목차             7387 non-null   object
 10  주제용어           5241 non-null   object
 11  용어사전           13008 non-null  object
 12  주제분류           13008 non-null  object
 13  combined_text  13008 non-null  object
dtypes: object(14)
memory usage: 1.5+ MB


In [ ]:
# 모든 값이 결측값인 행 삭제
data = data.dropna(how='all')

data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13008 entries, 1 to 17093
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   메타데이터          13008 non-null  object
 1   자료상태           13008 non-null  object
 2   연구과제번호         13008 non-null  object
 3   결과물 구분         12817 non-null  object
 4   자료유형           13008 non-null  object
 5   저작             13008 non-null  object
 6   서명             13008 non-null  object
 7   주초록            7759 non-null   object
 8   메타작성자초록        12303 non-null  object
 9   목차             7387 non-null   object
 10  주제용어           5241 non-null   object
 11  용어사전           13008 non-null  object
 12  주제분류           13008 non-null  object
 13  combined_text  13008 non-null  object
dtypes: object(14)
memory usage: 1.5+ MB


In [ ]:
# 모든 열에 대해서 결측값을 빈 문자열로 대체
data = data.fillna('')

data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13008 entries, 1 to 17093
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   메타데이터          13008 non-null  object
 1   자료상태           13008 non-null  object
 2   연구과제번호         13008 non-null  object
 3   결과물 구분         13008 non-null  object
 4   자료유형           13008 non-null  object
 5   저작             13008 non-null  object
 6   서명             13008 non-null  object
 7   주초록            13008 non-null  object
 8   메타작성자초록        13008 non-null  object
 9   목차             13008 non-null  object
 10  주제용어           13008 non-null  object
 11  용어사전           13008 non-null  object
 12  주제분류           13008 non-null  object
 13  combined_text  13008 non-null  object
dtypes: object(14)
memory usage: 1.5+ MB


#### RoBERT 모델링

In [ ]:
# 레이블 인코딩
label_encoder = LabelEncoder()

data['주제분류'] = label_encoder.fit_transform(data['주제분류'])

# 데이터셋 분리(8:2)
#train_texts1, val_texts1, train_labels1, val_labels1 = train_test_split(data['저작'], data['주제분류'], test_size=0.2, random_state=42)
#train_texts2, val_texts2, train_labels2, val_labels2 = train_test_split(data['서명'], data['주제분류'], test_size=0.2, random_state=42)
#train_texts3, val_texts3, train_labels3, val_labels3 = train_test_split(data['주초록'], data['주제분류'], test_size=0.2, random_state=42)
#train_texts4, val_texts4, train_labels4, val_labels4 = train_test_split(data['메타작성자초록 '], data['주제분류'], test_size=0.2, random_state=42)
#train_texts5, val_texts5, train_labels5, val_labels5 = train_test_split(data['목차'], data['주제분류'], test_size=0.2, random_state=42)
#train_texts6, val_texts6, train_labels6, val_labels6 = train_test_split(data['주제용어'], data['주제분류'], test_size=0.2, random_state=42)
#train_texts7, val_texts7, train_labels7, val_labels7 = train_test_split(data['용어사전'], data['주제분류'], test_size=0.2, random_state=42)
train_texts, val_texts, train_labels, val_labels = train_test_split(data['combined_text'], data['주제분류'], test_size=0.2, random_state=42)

#### 저작

In [ ]:
# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-large')

# 토큰화 함수 정의
def tokenize_function(texts):
    return tokenizer(texts, padding='max_length', truncation=True, max_length=128)

# 데이터셋 토큰화
train_encodings = tokenize_function(train_texts.tolist())
val_encodings = tokenize_function(val_texts.tolist())

# PyTorch 데이터셋 클래스 정의
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# 데이터셋 생성
train_dataset = CustomDataset(train_encodings, train_labels.tolist())
val_dataset = CustomDataset(val_encodings, val_labels.tolist())

# 모델 로드

In [ ]:
from adabelief_pytorch import AdaBelief
from transformers import get_linear_schedule_with_warmup

# 모델 로드
model = AutoModelForSequenceClassification.from_pretrained('klue/roberta-large', num_labels=len(label_encoder.classes_))

# 트레이너 설정
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',  # 평가 전략을 epoch으로 설정
    save_strategy='epoch',  # 저장 전략을 평가 전략과 동일하게 설정
    logging_strategy='steps',  # 로깅 전략 설정
    logging_steps=10,  # 몇 스텝마다 로깅할지 설정
    logging_dir='./logs',  # 로그 파일 저장 디렉토리
    per_device_train_batch_size=32,  # 배치 크기 증가
    per_device_eval_batch_size=32,
    num_train_epochs=3,  # 에폭 수 증가
    weight_decay=0.01,
    learning_rate=3e-5,  # 학습률 조정
    report_to="none",  # 로그를 어디에 보고할지 설정 (기본은 "none", "tensorboard", "wandb" 등 가능)
    load_best_model_at_end=True,  # 최적의 모델을 마지막에 로드
    metric_for_best_model="eval_loss",  # Early Stopping을 위한 평가 지표 설정
    greater_is_better=False  # 평가 지표가 낮을수록 좋은 경우 False로 설정
)

# Early Stopping Callback 설정(조기종료)
early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=3)  # patience는 개선이 없는 에폭 수를 의미


# AdaBelief 최적화 알고리즘 설정
optimizer = AdaBelief(model.parameters(), lr=3e-5, eps=1e-8, weight_decay=0.01, rectify=False)

# 학습률 스케줄러 설정
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(train_dataset) * training_args.num_train_epochs
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    optimizers=(optimizer, scheduler)  # 최적화 알고리즘 및 스케줄러 설정
)


# #백업
# model = AutoModelForSequenceClassification.from_pretrained('klue/roberta-large', num_labels=len(label_encoder.classes_))

# # 트레이너 설정
# training_args = TrainingArguments(
#     output_dir='./results',
#     evaluation_strategy='epoch',
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=1,
#     weight_decay=0.01,
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Please check your arguments if you have upgraded adabelief-pytorch from version 0.0.5.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  ---------
adabelief-pytorch=0.0.5  1e-08  False              False
>=0.1.0 (Current 0.2.0)  1e-16  True               True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended eps = 1e-8                                      Recommended eps = 1e-16
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.

Weight decoupling enabled in AdaBelief


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,3.839000,3.748749
2,3.123100,2.949635
3,2.526200,2.586785


TrainOutput(global_step=978, training_loss=3.400201239712643, metrics={'train_runtime': 1151.1401, 'train_samples_per_second': 27.119, 'train_steps_per_second': 0.85, 'total_flos': 7276747948056576.0, 'train_loss': 3.400201239712643, 'epoch': 3.0})

In [ ]:
# 예측 함수 정의 (디바이스 설정 추가)
def predict(text):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    inputs = tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=128).to(device)
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)
    return label_encoder.inverse_transform(predictions.cpu().numpy())

In [ ]:
# 예측 예시
example_text = "프로이트"
predicted_label = predict(example_text)
print(f"예측된 주제분류: {predicted_label[0]}")

예측된 주제분류: 서양철학


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch

# 모델 평가 함수 정의
def evaluate_model(trainer, val_dataset):
    predictions, labels, _ = trainer.predict(val_dataset)
    predictions = torch.tensor(predictions)  # numpy.ndarray를 torch.Tensor로 변환
    predictions = torch.argmax(predictions, dim=-1)
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    return accuracy, precision, recall, f1

# 모델 평가
accuracy, precision, recall, f1 = evaluate_model(trainer, val_dataset)
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.4539
Precision: 0.2895
Recall: 0.4539
F1 Score: 0.3287


In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score

label_encoder = LabelEncoder()
data['주제분류'] = label_encoder.fit_transform(data['주제분류'])

# 각 컬럼의 영향도 분석
columns = ['저작', '서명', '주초록', '메타작성자초록 ', '목차', '주제용어', '용어사전']
influence_scores = {}

for column in columns:
    X = data[column].apply(lambda x: len(str(x)))  # 텍스트 길이로 대체
    y = data['주제분류']

    model = LinearRegression()
    model.fit(X.values.reshape(-1, 1), y)

    y_pred = model.predict(X.values.reshape(-1, 1))
    influence_scores[column] = r2_score(y, y_pred)

# 결과 출력
for column, score in influence_scores.items():
    print(f"{column}: {score:.4f}")

저작: 0.0058
서명: 0.0157
주초록: 0.0006
메타작성자초록 : 0.0011
목차: 0.0002
주제용어: 0.0000
용어사전: 0.0127
